In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.options.display.max_rows = 200
pd.options.display.max_columns = 200

In [3]:
class Prepack:
    """Data preparation library v0.1""" 
    
    def __init__(self):
        pass
    
    @staticmethod
    def isnan(v):
        import math as m
        if isinstance(v, float) and m.isnan(v):
            return True
        else:
            return False
    
    @staticmethod    
    def list_concat(lst1, lst2):

        lst3 = []
        size1 = len(lst1)
        size2 = len(lst2)
        #здесь будет размер самого длинного списка
        size_max = max(size1,size2)

        #вытаскиваем и делаем строковыми значения из обоих списков
        for i in range(size_max):
            ss = ''
            if i < size1 and not Prepack.isnan(lst1[i]):
                s = str(lst1[i])
                if len(s) > 0:
                    ss += s
            if i < size2 and not Prepack.isnan(lst2[i]):
                s = str(lst2[i])
                if len(s) > 0:
                    if len(ss) > 0:
                        ss += ' '
                    ss += s
            #кладем все в список в виде объединенных значений
            lst3 += [ss]
        return lst3
    
    @staticmethod 
    def p2c(lst):
        import numpy as np
        length = len(lst)
        max_len = max(len(repr(el)) for el in lst)
        # делаем кортеж со значениями из списка
        cols = tuple(lst)
        # делаем массив из кортежа названий и диапазона от 0 до length, .T транспонирует матрицу
        a = np.array([np.arange(length),cols]).T

        #смотрим форму массива, по оси 0 - это y, и по оси 1 - это x
        y, x = a.shape

        #берем половину массив, округляя в большую сторону
        col1 = a[:int(np.ceil(y/2))].copy()
        #оставщаяся часть
        col2 = a[int(np.ceil(y/2)):].copy()

        #определяем максимальную длину колонок, чтобы увеличить одну из колонок, если она получилась короче, например, когда 
        #кол-во столбцов было изначально 3, первая колонка будет длиной 2, а вторая 1
        length = max(col1.shape[0], col2.shape[0])

        #меняем размер колонок по максимальной длине
        col1.resize([length,2])
        col2.resize([length,2])

        #Приходится изворачиваться, у Питона нет ++/-- для инкремента/декремента. Те мелочи, за которые я не люблю Питон 
        i = 0
        while i < length:
            c1 = str(col1[i][0]).ljust(3) #первая колонка
            c1_len = len(c1) 
            c2 = repr(col1[i][1]).ljust(max_len+3) 
            c2_len = len(c2) 
            c3 = str(col2[i][0]).ljust(3) #вторая колонка
            c3_len = len(c3)
            c4 = repr(col2[i][1])
            print(c1 + c2 + c3 + c4)
            i += 1
    
    #тут пара функций для удобного сохранения из загрузки 
    @staticmethod 
    def load(filepath):
        import pickle as pkl
        with open(filepath, "rb") as f:
            return pkl.load(f)
    
    @staticmethod 
    def save(data, filepath):
        import pickle as pkl
        with open(filepath, "wb") as f:
            return pkl.dump(data, f, 2) #2 is protocol version
    
    @staticmethod 
    def read_excels(filepath):
        import pandas as pd
        return pd.read_excel(filepath, sheet_name=None, header=None, na_filter=False)

    @staticmethod 
    def read_excel(filepath):
        import pandas as pd
        return pd.read_excel(filepath, header=None, na_filter=False)

    @staticmethod 
    def read_zip(filepath):
        import zipfile as zip
        z = zip.ZipFile(filepath, mode='r')
        names = tuple(z.namelist())
        lst = []
        for f in names:
            lst += [z.open(f)]
        return (names, tuple(lst))
    
    @staticmethod 
    def df_filter_and(df, fltr):
        import numpy as np
        lst = []
        for col in fltr:
            if fltr[col] == 'isnum':
                lst.append(df.loc[:,col].astype(str).str.replace('.','').str.isnumeric())
            elif fltr[col] == 'isnotnum':
                lst.append(~df.loc[:,col].astype(str).str.replace('.','').str.isnumeric())
            else:
                lst.append(df.loc[:,col] == fltr[col])
                
        if len(lst) == 1:
            return lst[0]
        else:
            res = lst[0]
            for i in range(1,len(lst)):
                res = np.logical_and(res, lst[i])
            return res
    
    @staticmethod 
    def df_filter_or(df, fltr):
        import numpy as np
        lst = []
        for col in fltr:
            if fltr[col] == 'isnum':
                lst.append(df.loc[:,col].astype(str).str.replace('.','').str.isnumeric())
            elif fltr[col] == 'isnotnum':
                lst.append(~df.loc[:,col].astype(str).str.replace('.','').str.isnumeric())
            else:
                lst.append(df.loc[:,col] == fltr[col])
        
        if len(lst) == 1:
            return lst[0]
        else:
            res = lst[0]
            for i in range(1,len(lst)):
                res = np.logical_or(res, lst[i])
            return res
        
    
pp = Prepack       

In [4]:
def parse_0507011(filepath):
    df = pp.read_excel(filepath)
    
    for col in df.columns:
        df.iloc[:,col] = df.iloc[:,col].astype(str).str.strip()

    cols = []

    # делаем назzвания столбцов в список
    for i in [8,9]:
        cols = pp.list_concat(cols, list(df.iloc[i]))

    for i, el in enumerate(cols):
        cols[i] = cols[i].replace('\n','')


    # данные только с 0 по 12 столбец
    df = df.iloc[:,0:12]

    # для подготовки маски фильтра будет использована специальная функция, которая создает маски по количеству переданных условий,
    # а потом складывает их через логическое И &. Т.е. остаются только те строки, которые удовлетворяют всем условиям вместе.
    # В данном случае у нас 2 условия, поле наименование не число, а поле Код число.
    f = pp.df_filter_and(df, {0: 'isnotnum', # Наименование
                              1: 'isnum',
                              6: 'isnum'}) # 'Код по бюджетной классификации'
    df_0 = df[f].reset_index(drop=True)
    
    df_0.iloc[:,6] = df_0.iloc[:,6].astype('float64') 
    
    new = df_0[2].str.extract(r'(\d\d)(\d\d)')
    new.columns = ['РЗ','ПР']
    df_0 = df_0.join(new)
    f = pp.df_filter_and(df_0, {3: '', # 'Коды ВР' 
                                4: '', # 'Коды ЦСР'
                                6: 'isnum', # 'Уточненная сводная бюджетная роспись'
                                'ПР': '00'})
    df_0 = df_0[f]
    df_0.iloc[:,[0,6,12]]
    year = df_0.iloc[:,[0,6,12]].groupby(['РЗ']).agg({0: 'first', 6: 'sum'}).reset_index()
    year.columns = ['РЗ','Наименование','Сумма']
    return year

In [10]:
yy = {}
for y in range(2,19+1):
    year = str(2000 + y)
    yy[year] = parse_0507011(year + '.xls')

pp.save(yy, '02-19.pkl')

,РЗ,Наименование,Сумма
0,01,Общегосударственные вопросы,1.445454e+09
1,02,Национальная оборона,1.001196e+09
2,03,Национальная безопасность и правоохранительная...,1.440675e+09
3,04,Национальная экономика,2.759918e+09
4,05,Жилищно-коммунальное хозяйство,2.788406e+08
5,06,Охрана окружающей среды,1.958698e+08
6,07,Образование,8.708491e+08
7,08,"Культура, кинематография",1.396592e+08
8,09,Здравоохранение,7.147411e+08
9,10,Социальная политика,4.839450e+09
